# Preparamos los datos para nuestro script


In [1]:
import urllib, json, datetime, pycurl, requests
from datetime import date
import pandas as pd 
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup #Web scrapping


In [2]:
# Montamos la URL de donde recibiremos nuestra IP pública:
url_ip="http://ip-api.com/json/?fields=status,message,continent,continentCode,country,countryCode,region,regionName,city,district,zip,lat,lon,timezone,offset,currency,isp,org,as,asname,reverse,mobile,proxy,hosting,query"

In [3]:
# Obtenemos la fecha del día en curso:
fecha = date.today()
print(fecha)

2020-10-21


# Obtenemos la IP pública de nuestra máquina con WebScrapping


In [4]:
# Requests conecta con la web y devuelve <Response [200]> si ha conectado correctamente. Lo almacenamos en response.
try:
    response = requests.get(url_ip)
except:
    response = -1000
    print("No se puede comunicar con la web!!")
#print(response)

In [5]:
# Comprobamos si la conexión devuelve el valor de correcta conexión '<Response [200]>'
respuesta=str(response).find("<Response [200]>")
if respuesta != -1:
    print("Se ha conectado correctamente")
else:
    print("Ha habido un error")

Se ha conectado correctamente


In [6]:
# Obtenemos el contenido de la url:
sopa = requests.get(url_ip)

In [7]:
sopa

<Response [200]>

In [8]:
# Tratamos los datos como json
sopa_dict = dict(json.loads(sopa.content))
sopa_dict

{'status': 'success',
 'continent': 'Europe',
 'continentCode': 'EU',
 'country': 'Spain',
 'countryCode': 'ES',
 'region': 'MD',
 'regionName': 'Madrid',
 'city': 'Getafe',
 'district': '',
 'zip': '28906',
 'lat': 40.3057,
 'lon': -3.7329,
 'timezone': 'Europe/Madrid',
 'offset': 7200,
 'currency': 'EUR',
 'isp': 'Telefonica de Espana SAU',
 'org': 'RIMA (Red IP Multi Acceso)',
 'as': 'AS3352 TELEFONICA DE ESPANA',
 'asname': 'Telefonica_de_Espana',
 'reverse': '26.red-83-42-158.dynamicip.rima-tde.net',
 'mobile': False,
 'proxy': False,
 'hosting': False,
 'query': '83.42.158.26'}

In [9]:
print("Podemos ver que nos muestra el valor de la ciudad, que en mi caso es "+sopa_dict['city']+".")
print(sopa_dict['city']+" pertenece a la provincia de "+sopa_dict['regionName']+".")

Podemos ver que nos muestra el valor de la ciudad, que en mi caso es Getafe.
Getafe pertenece a la provincia de Madrid.


In [15]:
# Obtenemos los valores que necesitamos para el tiempo
ciudad = sopa_dict['city']
region = sopa_dict['regionName']
ip_publica = sopa_dict['query']
LAT = sopa_dict['lat']
LON = sopa_dict['lon']
ISP = sopa_dict['isp']

print("la ciudad es "+ciudad)
print("La región es "+region)
print("La ip pública es "+ip_publica)
print("La latitud es "+str(LAT))
print("La longitud es "+str(LON))
print("El ISP es "+ISP)

la ciudad es Getafe
La región es Madrid
La ip pública es 83.42.158.26
La latitud es 40.3057
La longitud es -3.7329
El ISP es Telefonica de Espana SAU


## El tiempo

In [16]:
# La URL de la API
url_tiempo = "https://api.climacell.co/v3/weather/forecast/hourly"